# In this notebook we generate the six functions that will be consulted in the API.

## Modules needed

In [3]:
import pandas as pd
import numpy as np

## Root Function

In [88]:
def root():
    f = open("APIData/welcome.txt", "r")
    return print(f.read())

In [89]:
root()

Hi! Welcome to the Henry Individial Project - Machine Learning Operations

The objective of this API is to make available some information of the Steam Games Datasets. The Raw Data, alongside with all the 
processes can be found in: https://github.com/juanesfco/HENRY .

This API contains 7 functions detailed below. The possible inputs of this functions can be seen in: 

1. User Data Function: This function takes as input the user id and returns the amount of money spent by the user, the 
                        recommendation percentage and the number of games bought.

How to use it?
If you want to know information about user_id="Riot-Punch". The request will be: 

henryplmlops.onrender.com/userdata/?user_id=Riot-Punch

2. Count Reviews Function: This function takes as input two dates and returns the amount of users that made reviews 
                            within those dates and recommendation percentage.

How to use it?
If you want to know information within date1="2014-03-05" a

## User Data function

This function takes as input the user id and returns the amount of money spent by the user, the recommendation percentage and the number of games bought.

In [22]:
def userdata(user_id: str):

    ## Step 1. Import General User Items dataframe and filter using user id input
    path = 'APIData/'
    fn_userItems = path + 'df_userItems.csv'
    df_userItems = pd.read_csv(fn_userItems)
    df_userItems_i = df_userItems[df_userItems['user_id']==user_id]

    # Check if user id is valid
    if len(df_userItems_i) == 0:
        cantidad = 0
        cantidadItems = 0
    else:
        ## Step 2. Save number of items
        cantidadItems = df_userItems_i['items_count'].values[0]

        # Check if the user has items
        if cantidadItems == 0:
            cantidad = 0
        else:
            ## Step 3. Import Specific User Items dataframe
            steam_id = df_userItems_i['steam_id'].values[0]
            pathItems = path + 'ItemsData/'
            fni = pathItems + 'itemsData_' + str(steam_id) + '.csv'
            dfi = pd.read_csv(fni)

            ## Step 4. Import Steam Games Dataframe
            fn_steamGames = path + 'df_steamGames.csv'
            df_steamGames = pd.read_csv(fn_steamGames)

            ## Step 5. Calculate money spent iterating over item ids and looking for them in Steam Games Dataframe
            I_IDS = dfi['item_id'].values
            cantidad = 0
            for i in I_IDS:
                try:
                    p = df_steamGames[df_steamGames['id']==i]['price'].values[0]
                except:
                    p = 0 # If game is not found, assign price of 0
                cantidad += p
        
    ## Step 6. Import General User Reviews Data
    fnReviews = path + 'df_reviews.csv'
    dfReviews = pd.read_csv(fnReviews)
    dfReviews_i = dfReviews[dfReviews['user_id']==user_id]

    # Check if user id is valid
    if len(dfReviews_i) == 0:
        r = 0
    else:
        steam_id = dfReviews_i['steam_id'].values[0]
        ## Step 7. Import Specific User Reviews Data
        fnReviews_i = path + 'ReviewsData/revData_' + str(steam_id) + '.csv'
        try:
            dfReviews_i = pd.read_csv(fnReviews_i)

            ## Step 8. Calculate recommendation percentage
            n = len(dfReviews_i)
            s = sum(dfReviews_i['recommend'])
            r = n/s*100
        except:
            r=0
    
    return {"cantidad":cantidad.item(),"items":cantidadItems.item(),"recper":r}

In [23]:
userdata('Riot-Punch')

{'cantidad': 3427.119999999979, 'items': 328, 'recper': 0}

## Count Reviews Function

This function takes as input two dates and returns the amount of users that made reviews within those dates and recommendation percentage.

In [72]:
def countreviews(date1: str, date2: str):
    date1 = pd.to_datetime(date1)
    date2 = pd.to_datetime(date2)

    fn_reviews_r = 'APIData/df_reviews_r.csv'
    df_reviews_r = pd.read_csv(fn_reviews_r)

    df_reviews_r_betweenDates = df_reviews_r[pd.to_datetime(df_reviews_r['posted']).between(date1,date2)]
    n = len(df_reviews_r_betweenDates)

    users = len(df_reviews_r_betweenDates.groupby('steam_id').count())

    if n > 0:
        r = sum(df_reviews_r_betweenDates['recommend'])
        p = r/n*100
    else:
        p=0

    return {"UserCount":users,"RecPercentage":p}    

In [67]:
countreviews('2014-03-05','2014-04-05')

{'UserCount': 1473, 'RecPercentage': 91.99057714958775}

## Genre Function

This function takes as input the genre and returns the rank based on `PlayTimeForever` column

In [32]:
def genre(genre: str):
    path = 'APIData/'
    fn_genresRank = path + 'genresRank.csv'
    df_genresRank = pd.read_csv(fn_genresRank)

    genreRow = df_genresRank[df_genresRank['genre'] == genre]
    if len(genreRow) == 0:
        return('Not a genre (case sensitive)')
    else:
        rank = genreRow.index[0] + 1
        return {"rank": rank.item()}

In [33]:
genre('Action')

{'rank': 1}

## User for Genre Function

This function takes as input the genre and returns the top 5 user rank based on `PlayTimeForever` column, with his URL and user_id

In [10]:
def userforgenre(genre: str):
    path = 'APIData/GenresData/'
    fn_genreRank = path + 'genreData_' + genre + '.csv'
    try:
        df_genreRank = pd.read_csv(fn_genreRank)
        top5 = df_genreRank.loc[0:4,'steam_id'].values
        fn_userItems = 'APIData/df_userItems.csv'
        df_userItems = pd.read_csv(fn_userItems)

        df_top5 = df_userItems[df_userItems['steam_id'].isin(top5)].drop(columns=['items_count','steam_id'])
        
        if len(df_genreRank) == 0:
            return('No Playing time')
        else:
            return {"top5": df_top5}
    except:
        return('Not a genre (case sensitive)')

In [11]:
userforgenre('Dark')

{'top5':                  user_id                                           user_url
 4915    1001101001110101      http://steamcommunity.com/id/1001101001110101
 10898            otheldo               http://steamcommunity.com/id/otheldo
 41509  76561198018684159  http://steamcommunity.com/profiles/76561198018...
 43928      MaeveOfSwords         http://steamcommunity.com/id/MaeveOfSwords
 75278  76561198084332433  http://steamcommunity.com/profiles/76561198084...}

## Developer Function

This function takes as input the developer and returns the amount of items and the percentage of free content by the develope

In [79]:
def developer(dev:str):
    fn_steamGames = 'APIData/df_steamGames.csv'
    df_steamGames = pd.read_csv(fn_steamGames)
    df_steamGames_dev = df_steamGames[df_steamGames['developer'] == dev]
    n = len(df_steamGames_dev)
    if n == 0:
        return('No games found of this developer')
    else:
        df_steamGames_dev.loc[:,'release_date'] = pd.to_datetime(df_steamGames_dev['release_date']).dt.year
        year = df_steamGames_dev.groupby('release_date').count().index.values
        count  = df_steamGames_dev.groupby('release_date').count()['price'].values
        df_steamGames_dev.loc[:,'price'] = (df_steamGames_dev['price'] == 0)
        free = df_steamGames_dev.groupby('release_date').sum()['price'].values
        per = (free/count*100).astype(str)
        df_ret = pd.DataFrame({'year':year.tolist(),'count':count.tolist(),'free_content':per})
        df_ret['free_content']= df_ret['free_content'].str.slice(0,4)
        df_ret['free_content'] = (df_ret['free_content'].values + '%').tolist()
        return {dev:df_ret.to_dict()}


In [80]:
developer('Sacada')

{'Sacada': {'year': {0: 2017, 1: 2018},
  'count': {0: 17, 1: 1},
  'free_content': {0: '0.0%', 1: '0.0%'}}}

## Sentiment Analysis Function

This function takes as input a year and returns a list of sentiment analysis of reviews of games 

In [40]:
def sentiment_analysis(year:int):
    fn_steamGames = 'APIData/df_steamGames.csv'
    df_steamGames = pd.read_csv(fn_steamGames)
    df_steamGames['release_date'] = pd.to_datetime(df_steamGames['release_date']).dt.year
    ids = df_steamGames[df_steamGames['release_date']==year]['id'].values

    fn_reviews_r = 'APIData/df_reviews_r.csv'
    df_reviews_r = pd.read_csv(fn_reviews_r)
    df_reviews_r_ids = df_reviews_r[df_reviews_r['item_id'].isin(ids)]
    sa_count = df_reviews_r_ids.groupby('sentiment_analysis').count()['item_id'].values
    sa_index = df_reviews_r_ids.groupby('sentiment_analysis').count().index.values

    try: 
        sa_index = np.char.replace(np.char.replace(np.char.replace(sa_index.astype(str),'0','Negative'),'1','Neutral'),'2','Positive')
    except:
        pass
    rdic = {'Negative':0,'Neutral':0,'Positive':0}
    for i in range(len(sa_index)):
        rdic[sa_index[i]] = sa_count[i].item()
    
    return rdic

In [41]:
sentiment_analysis(2016)

{'Negative': 881, 'Neutral': 2455, 'Positive': 2870}

## Game Recommendation Fucntion

This function takes as input a game id and returns 5 similar games based on a machine learning model.

In [16]:
def recomendacion_juego(item_id:int):
    fn_steamGames = 'APIData/df_steamGames.csv'
    df_steamGames = pd.read_csv(fn_steamGames)

    df_gamesNameId = df_steamGames[['id','app_name']]
    i = df_gamesNameId[df_gamesNameId['id'] == float(item_id)].index[0]
    name = df_gamesNameId[df_gamesNameId['id'] == float(item_id)]['app_name'].values[0]

    fn_top5 = 'APIData/MLData/df_top5.csv'
    df_top5 = pd.read_csv(fn_top5)
    df_top5_i = df_top5.loc[:,str(i)].values

    rec_names = df_gamesNameId.loc[df_top5_i,'app_name'].values


    return {"Based on game:":name,"We recommend:":list(rec_names)}

In [17]:
recomendacion_juego(643980.0)

{'Based on game:': 'Ironbound',
 'We recommend:': ['Aurora Trail',
  'Labyrinth - Starter Pack',
  'MINDNIGHT',
  'Immortal Empire - Starter Pack 2',
  'Immortal Empire - Starter Pack 1']}